In [24]:
from itertools import permutations
from collections import defaultdict
def read_and_process_file(filename='input'):
    file1 = open(f'{filename}_16', 'r')
    Lines = [l.rstrip() for l in file1.readlines()]
    flows = {}
    connections = {}
    for l in Lines:
        current, l = l.replace("Valve ", "").split(" has flow rate=")
        flow, cnx = l.split("; ")
        flow = int(flow)
        if flow > 0:
            flows[current] = flow
        connections[current] = [c.rstrip().lstrip() for c in cnx.replace("tunnels lead to valves ", "").replace("tunnel leads to valve ","").split(",")]
    return flows, connections

In [180]:
def create_graph(connections):
    graph = {}
    for c, neighbors in connections.items():
        graph[c] = defaultdict(lambda: 999999999)
        for n in neighbors:
            graph[c][n] = 1
    
    all_nodes = connections.keys()
    for n1 in all_nodes:
        for n2 in all_nodes:
            for n3 in all_nodes:
                graph[n2][n3] = min(graph[n2][n3], graph[n2][n1] + graph[n1][n3])
    return graph

In [189]:
def calculate_benefit(path, flows, graph, time_limit):
    benefit = 0
    path = path.copy()
    current = path.pop(0)
    for next_step in path:
        time_limit -= graph[current][next_step] + 1 #go and open
        #print(f"New benefit: {current}-{next_step} adds {flows[next_step]*max(0,time_limit)}")
        benefit += flows[next_step]*max(0,time_limit)
        if time_limit <0:
            return benefit
        current = next_step
    return benefit

In [190]:
def find_all_combinations(current, all_options, graph, time_limit):
    current_node = current[-1]
    for next_step in all_options:
        if next_step in current:
            continue
        #print(f"Outer: {current} | time: {time_limit}")
        new_time_limit = time_limit - (graph[current_node][next_step] + 1)
        if new_time_limit >= 0:
            #print(f'Adding new combination: {current+[next_step]} (time remaining: {new_time_limit}) | nodes remaining: {remaining}')
            combinations.append(current+[next_step])
            find_all_combinations(current.copy()+[next_step], all_options, graph, new_time_limit)
        else:
            continue

In [262]:
flows, connections = read_and_process_file('input')
graph = create_graph(connections)

#### One star

In [263]:
combinations = []
flows["AA"] = 0
time = 30

find_all_combinations(['AA'], list(flows.keys()), graph, time)
res = max([calculate_benefit(c, flows, graph, time) for c in combinations])
print(f"The benefit in {time}min is: {res}")

The benefit in 30min is: 2265


#### Two stars

In [264]:
combinations = []
def find_best_paths(flows, graph, time):
    find_all_combinations(['AA'], list(flows.keys()), graph, time)
    best_res = 0
    
    for c1 in combinations:
        for c2 in combinations:
            if len(list(set(c1+c2))) == (len(c1)+len(c2)-1):
                best_res = max(best_res, calculate_benefit(c1, flows, graph, time)+calculate_benefit(c2, flows, graph, time))
            else:
                continue
    return best_res

In [265]:
def get_key(c):
    return " ".join(sorted(c))

def check_keys_unique(c1, c2):
    c1 = c1.split(" ")
    c2 = c2.split(" ")
    return len(list(set(c1+c2))) == (len(c1)+len(c2)-1)

In [266]:
combinations = []
def find_best_paths(flows, graph, time):
    find_all_combinations(['AA'], list(flows.keys()), graph, time)
    best_res = 0
    
    combination_gain = defaultdict(int)
    for c in combinations:
        combination_gain[get_key(c)] = max(combination_gain[get_key(c)], calculate_benefit(c, flows, graph, time))
    
    combination_sets = list(combination_gain.keys())
    for c1 in combination_sets:
        for c2 in combination_sets:
            if check_keys_unique(c1,c2):
                best_res = max(best_res, combination_gain[c1]+combination_gain[c2])
    return best_res

In [267]:
res = find_best_paths(flows, graph, 26)
print(f"The benefit in {time} min using 2 explorers is: {res}")

The benefit in 30 min using 2 explorers is: 2811


#### First ideas (worse)

In [ ]:
def BFS_SP(graph, start, goal):
    explored = []
     
    # Queue for traversing the
    # graph in the BFS
    queue = [[start]]
     
    # If the desired node is
    # reached
    if start == goal:
        #print("Same Node")
        return 0
     
    # Loop to traverse the graph
    # with the help of the queue
    while queue:
        path = queue.pop(0)
        node = path[-1]
         
        # Condition to check if the
        # current node is not visited
        if node not in explored:
            neighbours = graph[node]
             
            # Loop to iterate over the
            # neighbours of the node
            for neighbour in neighbours:
                new_path = list(path)
                new_path.append(neighbour)
                queue.append(new_path)
                 
                # Condition to check if the
                # neighbour node is the goal
                if neighbour == goal:
                    #print("Shortest path = ", *new_path)
                    return len(new_path)-1
            explored.append(node)
 
    # Condition when the nodes
    # are not connected
    print(f"So sorry, but a connecting path doesn't exist {start} - {goal}")
    return -1

In [99]:
flows, connections = read_and_process_file('input')
flows = {k:v for k,v in sorted(flows.items(), key=lambda item: item[1], reverse=True) if v>0}

In [102]:
res = 0

#current_node - remaining_nodes - _time - _res

current_node = "AA"
remaining_nodes = list(flows.keys())
_time = 30
_res = 0

def explore(current_node, remaining_nodes, _time, _res):
    best_res = _res
    if remaining_nodes is None:
        return best_res
    for n in remaining_nodes:
        time_required = BFS_SP(connections, current_node, n)
        _new_time = _time - time_required - 1
        if _new_time <= 0:
            best_res = max(_res,best_res)
        else:
            new_remaining = remaining_nodes.copy()
            new_remaining.remove(n)
            best_res = max(best_res, explore(n, new_remaining, _new_time, _res+(flows[n]*(_new_time))))
    return best_res
explore(current_node, remaining_nodes, _time, _res)

2265

In [ ]:
#2228 - too low

In [ ]:
res = 0
best_path = ""
case = 0
for p in permutations(flows.keys(), 15):
    case += 1
    if (case%100000)==0:
        print(f"Done ({case})")
    per_res = 0
    current = "AA"
    time = 30
    for n in p:
        time_required = BFS_SP(connections, current, n)
        time -= time_required #travel
        time -= 1 #open valve
        if time<0:
            break
        per_res += flows[n]*time
        current = n
    if res < per_res:
        res = per_res
        best_path = p
res